<img src="https://static.wixstatic.com/media/a9ca5e_825bd4d39e7d468faf735b801fa3dea4~mv2.png/v1/fill/w_1458,h_246,al_c,usm_0.66_1.00_0.01/a9ca5e_825bd4d39e7d468faf735b801fa3dea4~mv2.png" width="200">


 __Notebook voor het importeren van APEX Legends data__

 __Auteur:__ Melvin Folkers (Yellow Stacks B.V.)<br>
 __Versie:__ 12 april 2020<br>
 ***

 

# parameters

## import libraries

In [1]:
import os
import pandas as pd
import logging
import requests
import json

## notebook settings

In [3]:
from IPython.core.display import display, HTML
from IPython.core.interactiveshell import InteractiveShell
from IPython.display import Image

display(HTML("<style>.container { width:100% !important; }</style>")) # maakt de jupyter notebook cellen 100% breed
InteractiveShell.ast_node_interactivity = "all" # om meerdere grafieken/tabellen in 1 cel te kunnen laten zien.
pd.options.display.max_columns = None # laat alle kolommen zien

%matplotlib inline 

logger = logging.getLogger()
logger.setLevel(logging.INFO)

## Functies

In [4]:
class SessionParameters():

    def __init__(self, api_key, platform_id, user_id):

        self.api_key = api_key
        self.platform_id = platform_id
        self.user_id  = user_id

        self.url = f'https://public-api.tracker.gg/v2/apex/standard/profile/{self.platform_id}/{self.user_id}'
        self.headers = {'Content-Type': 'text/json',
                       'Accept-Charset': 'utf-8',
                       'TRN-Api-Key': self.api_key}

In [5]:
def request_data(params):
    response = requests.get(params.url, headers = params.headers)
    text = response.text
    json_data = json.loads(text)
    tables = json_data.get('data','')
    logging.info('aanwezige tabellen:\n {}'.format(', '.join([k for k,_v in tables.items()])))
    
    return tables

In [6]:
def get_userinfo(tables):
    userinfo = tables.get('userInfo')
    userinfo.pop('socialAccounts')
    user = pd.DataFrame(userinfo, index = [0])
    
    return user

In [7]:
def get_segments(tables):
    ttl = pd.DataFrame()

    for segment in tables.get('segments'):

        stats = get_stats(segment)

        ttl = pd.concat([ttl, stats], axis = 0, sort = False)
    
    return ttl
    

In [8]:
def get_stats(segment):
    
    stats = segment.get('stats')
    statlist = [x for x, _y in stats.items()]
    d = dict()
    
    for x in statlist:
        data = stats.get(x)

        d['{} #'.format(data['displayName'])] = data['value']
        d['{} %'.format(data['displayName'])] = data['percentile']
    
    df = pd.DataFrame(d, index = [0])
    df['segment_name'] =  segment.get('metadata').get('name')
    df['segment_img'] = segment.get('metadata').get('tallImageUrl')
    df['segment_type'] = segment.get('type')
    return df
    

# Gebruiker authenticeren

In [11]:
api_key = os.environ.get('APEX_API_KEY')
platform_id = 'xbl'
user_id = 'cancan91'

In [12]:
params = SessionParameters(api_key, platform_id, user_id)

# Data opvragen van user 

In [13]:
tables = request_data(params)

INFO:root:aanwezige tabellen:
 platformInfo, userInfo, metadata, segments, availableSegments, expiryDate


In [14]:
platform = pd.DataFrame(tables.get('platformInfo'), index = [0])
avatar_url = platform['avatarUrl'][0]

In [15]:
display(HTML('<h1> data bekijken van {}</h1>'.format(platform['platformUserId'][0])))
Image(url= avatar_url, width=200, height=200)

In [16]:
tables.get('metadata')

{'currentSeason': 2, 'activeLegend': 'legend_9', 'activeLegendName': 'Octane'}

In [17]:
user = get_userinfo(tables)

In [18]:
user

,isPremium,isVerified,isInfluencer,countryCode,customAvatarUrl,customHeroUrl
0,False,False,False,None,None,None


In [19]:
metadata = pd.DataFrame(tables.get('metadata'), index = [0])

In [20]:
metadata

,currentSeason,activeLegend,activeLegendName
0,2,legend_9,Octane


In [21]:
segments = get_segments(tables)

In [23]:
segments

,Level #,Level %,Kills #,Kills %,Rank Score #,Rank Score %,Season 3 Wins #,Season 3 Wins %,Season 4 Wins #,Season 4 Wins %,segment_name,segment_img,segment_type,Stim Distance Travelled #,Stim Distance Travelled %,NOX: Gas Damage Dealt #,NOX: Gas Damage Dealt %,Rifts: Squadmates Phased #,Rifts: Squadmates Phased %
0,182.0,84.0,199.0,60.0,1190.0,72.0,5.0,90,1.0,84.0,Lifetime,None,overview,NaN,NaN,NaN,NaN,NaN,NaN
0,NaN,NaN,88.0,53.0,NaN,NaN,NaN,NaN,1.0,87.0,Octane,https://trackercdn.com/cdn/apex.tracker.gg/leg...,legend,26040.0,93.0,NaN,NaN,NaN,NaN
0,NaN,NaN,79.0,66.0,NaN,NaN,5.0,95.2,NaN,NaN,Caustic,https://trackercdn.com/cdn/apex.tracker.gg/leg...,legend,NaN,NaN,3476.0,93.0,NaN,NaN
0,NaN,NaN,32.0,37.0,NaN,NaN,0.0,None,NaN,NaN,Wraith,https://trackercdn.com/cdn/apex.tracker.gg/leg...,legend,NaN,NaN,NaN,NaN,13.0,96.5


# Read statistics

In [437]:
for index, row in segments.iterrows():
    txt = '<br>'.join([ f'<i>{k}</i> :<b>{v}</b>' for k,v in row.items() if k.startswith('segment') == False])
    
    if row['segment_img']:
        Image(url= row['segment_img'], width=200, height=200)
    HTML('<font color = "green"><big><b>{} - {}</b></big></font><br><br>{}'.format(row['segment_type'], row['segment_name'], txt))